## Simulated Error Channels ##
Shows the construction of the simulated error channels with associated Hinton diagrams.

All cells should execute in less than a couple of seconds.

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sbs

sbs.set(style="darkgrid")
%matplotlib inline

## Simulated Measurement Error Channel ##

In [ ]:
def gen_error_probs(
    error_arr_c : list, # Error distance weights
    error_arr_u : list, # Error state dependent weights up
    error_arr_d : list, # Error state dependent weights down
    n_qubits = 4):
    '''
        gen_error_probs
        
        Function to generate error probabilities for simulated measurement errors
        
        :: error_arr_c : list :: Weights on error sizes
        :: error_arr_u : list :: Weights on "up" state depentent errors
        :: error_arr_d : list :: Weights on "down" state depentent errors
        :: n_qubits    : int  :: Number of qubits
        
        Returns a measurement error channel
    
    '''
    
    # Build initial prob vector
    probs = [[0] * (2 ** n_qubits) for _ in range(2 ** n_qubits)]
    
    # Check size of weights
    if len(error_arr_c) != n_qubits + 1:
        raise Exception("Incorrect Error Array")
    
    if len(error_arr_u) != n_qubits + 1:
        raise Exception("Incorrect Error Array")
        
        
    if len(error_arr_d) != n_qubits + 1:
        raise Exception("Incorrect Error Array")
    
    # Fill vector
    for row in range(2 ** n_qubits):
        row_str = bin(row)[2:].zfill(n_qubits)


        for col in range(2 ** n_qubits):
            col_str = bin(col)[2:].zfill(n_qubits)

            diff_str = [i - j for i, j in zip(list(map(int, row_str)), list(map(int, col_str)))]   
            
            probs[row][col] += error_arr_u[sum(1 if i == -1 else 0 for i in diff_str)]
            probs[row][col] += error_arr_d[sum(1 if i == 1 else 0 for i in diff_str)]
            probs[row][col] += error_arr_c[n_qubits - sum(1 if i == 0 else 0 for i in diff_str)]
            
            probs[row][col] = max(0, probs[row][col])
            
    #Normalise rows, we can then do arbitrary scaling factors in the error arr
    for row, _ in enumerate(probs):
        np_row = np.array(probs[row])
        if sum(np_row) > 0:
            np_row = np_row / sum(np_row) 
        probs[row] = list(np_row)

    return probs

## Hinton Diagram Constructor Function ##

In [ ]:
# Based on code from matplotlib documentation #
# https://matplotlib.org/stable/gallery/specialty_plots/hinton_demo.html #

def hinton(matrix, max_weight=None, ax=None, n_qubits=4):
    """Draw Hinton diagram for visualizing a weight matrix."""
    ax = ax if ax is not None else plt.axes()

    if not max_weight:
        max_weight = 2 ** np.ceil(np.log2(np.abs(matrix).max()))

    ax.patch.set_facecolor('gray')
    ax.set_aspect('equal', 'box')

    for (x, y), w in np.ndenumerate(matrix):
        color = 'white' if w > 0 else 'black'
        size = np.sqrt(abs(w) / max_weight)
        rect = plt.Rectangle([x - size / 2, y - size / 2], size, size,
                             facecolor=color, edgecolor=color)
        ax.add_patch(rect)

    ax.autoscale_view()
    ax.invert_yaxis()
    
    # Square the plot
    plt.figsize=(6, 6)

    # Ticklabels
    ticks = [bin(i)[2:].zfill(n_qubits) for i in range(2 ** n_qubits)]

    ax.set_xticks([i for i in range(2 ** n_qubits)])
    ax.set_xticklabels(ticks, rotation=45)
    ax.xaxis.tick_top()

    ax.set_yticks([i for i in range(2 ** n_qubits)])
    ax.set_yticklabels(ticks, rotation=45)

    # Remove grid
    plt.grid(b=None)

    # Remove Ticks
    plt.tick_params(left=False, right=False, top=False, bottom=False)
    return ax

## Example Hinton Diagram ##

In [ ]:
n_qubits = 4
probs = gen_error_probs(
        [100, 5, 20, 1,0], # Const - Controls correlation of error weights
        [0, 5, 0, 5, 0], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0] # 0 -> 1 - Controls error biases
        )
ax = hinton(probs)

## Example Correlated Error Channels ##

In [ ]:
probs = gen_error_probs(
        [100, 10, 0,0,0], # Const - Controls correlation of error weights
        [0, 0, 0, 0, 0], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0] # 0 -> 1 - Controls error biases
        )

ax = hinton(probs)

In [ ]:
probs = gen_error_probs(
        [100, 0, 10,0,0], # Const - Controls correlation of error weights
        [0, 0, 0, 0, 0], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0] # 0 -> 1 - Controls error biases
        )

ax = hinton(probs)

In [ ]:
probs = gen_error_probs(
        [100, 0, 0, 10,0], # Const - Controls correlation of error weights
        [0, 0, 0, 0, 0], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0] # 0 -> 1 - Controls error biases
        )

ax = hinton(probs)

In [ ]:
probs = gen_error_probs(
        [100, 0, 0,0,10], # Const - Controls correlation of error weights
        [0, 0, 0, 0, 0], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0] # 0 -> 1 - Controls error biases
        )

ax = hinton(probs)

## Example State Dependent Error Channels ##

In [ ]:
probs = gen_error_probs(
        [100, 0, 0,0,0], # Const - Controls correlation of error weights
        [0, 10, 0, 0, 0], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0] # 0 -> 1 - Controls error biases
        )

ax = hinton(probs)

In [ ]:
probs = gen_error_probs(
        [100, 0, 0,0,0], # Const - Controls correlation of error weights
        [0, 0, 10, 0, 0], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0] # 0 -> 1 - Controls error biases
        )

ax = hinton(probs)

In [ ]:
probs = gen_error_probs(
        [100, 0, 0,0,0], # Const - Controls correlation of error weights
        [0, 0, 0, 10, 0], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0] # 0 -> 1 - Controls error biases
        )

ax = hinton(probs)

In [ ]:
probs = gen_error_probs(
        [100, 0, 0,0,0], # Const - Controls correlation of error weights
        [0, 0, 0, 0, 10], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0] # 0 -> 1 - Controls error biases
        )

ax = hinton(probs)